In [1]:
# !pip install tensorflow 
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 14.5 MB/s 
     |████████████████████████████████| 7.6 MB 63.3 MB/s 
     |████████████████████████████████| 182 kB 79.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import AutoTokenizer,AutoModelForSequenceClassification, TextClassificationPipeline
from transformers import Trainer, TrainingArguments
import numpy as np
import random
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [4]:
import re
# call the function
df_train = pd.read_excel('train.xlsx')
df_test =  pd.read_excel('test.xlsx')
df_valid = pd.read_excel('valid.xlsx')

df_train['Sentence'] = df_train['Sentence'].apply(lambda x: re.sub('per ','',x))
df_test['Sentence'] = df_test['Sentence'].apply(lambda x: re.sub('per ','',x))
df_valid['Sentence'] = df_valid['Sentence'].apply(lambda x: re.sub('per ','',x))



test_texts = list(df_test['Sentence'])
train_texts = list(df_train['Sentence'])
valid_texts = list(df_valid['Sentence'])

y= LabelEncoder()

train_labels = y.fit_transform(df_train['Emotion'])
valid_labels = y.fit_transform(df_valid['Emotion'])
test_labels = y.fit_transform(df_test['Emotion'])

target_names = list(df_train.Emotion.unique())


In [6]:

def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # safe to call this function even if cuda is not available
    if is_tf_available():
        import tensorflow as tf
        tf.random.set_seed(seed)

set_seed(1)

# the model we gonna train, base uncased BERT
# check text classification models here: https://huggingface.co/models?filter=text-classification
model_name = "xlm-roberta-base"
# max sequence length for each document/sentence sample
max_length = 512
# load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=True)




# tokenize the dataset, truncate when passed `max_length`, 
# and pad with 0's when less than `max_length`
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
valid_encodings = tokenizer(valid_texts, truncation=True, padding=True, max_length=max_length)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=max_length)


class NewsGroupsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

# convert our tokenized data into a torch Dataset
train_dataset = NewsGroupsDataset(train_encodings, train_labels)
valid_dataset = NewsGroupsDataset(valid_encodings, valid_labels)
test_dataset = NewsGroupsDataset(test_encodings, test_labels)
# load the model and pass to CUDA
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=7).to("cuda")


Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [8]:

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # calculate accuracy using sklearn's function
    acc = accuracy_score(labels, preds)
    return {
      'accuracy': acc,
    }

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=15,  # batch size per device during training
    per_device_eval_batch_size=15,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
#     logging_dir='./logs',            # directory for storing logs
#     load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
#     logging_steps=200,               # log & save weights each logging_steps
#     save_steps=200,
#     evaluation_strategy="steps",     # evaluate each `logging_steps`
)

trainer = Trainer(
    model=model,                         # the instantiated Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=valid_dataset,          # evaluation dataset
    compute_metrics=compute_metrics,     # the callback that computes metrics of interest
)
# train the model
trainer.train()
# evaluate the current model after training
trainer.evaluate()
# saving the fine tuned model & tokenizer

# model_path = "..output/vinai/phobert-base"
# model.save_pretrained(model_path)
# tokenizer.save_pretrained(model_path)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 5548
  Num Epochs = 5
  Instantaneous batch size per device = 15
  Total train batch size (w. parallel, distributed & accumulation) = 15
  Gradient Accumulation steps = 1
  Total optimization steps = 1850
  Number of trainable parameters = 278049031


Step,Training Loss
500,1.607700
1000,1.142400
1500,0.806600


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


***** Running Evaluation *****
  Num examples = 686
  Batch size = 15


{'eval_loss': 1.2470512390136719,
 'eval_accuracy': 0.6005830903790087,
 'eval_runtime': 3.5749,
 'eval_samples_per_second': 191.891,
 'eval_steps_per_second': 12.867,
 'epoch': 5.0}

In [9]:
from sklearn.metrics import classification_report
pred = trainer.predict(test_dataset)
y_pred = np.argmax(pred.predictions, axis=-1)
print(classification_report(test_labels,y_pred))

***** Running Prediction *****
  Num examples = 693
  Batch size = 15


              precision    recall  f1-score   support

           0       0.50      0.45      0.47        40
           1       0.60      0.58      0.59       132
           2       0.73      0.76      0.75       193
           3       0.62      0.70      0.65        46
           4       0.58      0.59      0.58       129
           5       0.75      0.73      0.74       116
           6       0.67      0.59      0.63        37

    accuracy                           0.66       693
   macro avg       0.63      0.63      0.63       693
weighted avg       0.66      0.66      0.66       693



In [10]:
trainer.save_model('trainer_roberta')
tokenizer.save_pretrained("tokenizer_roberta")

Saving model checkpoint to trainer-roberta
Configuration saved in trainer-roberta/config.json
Model weights saved in trainer-roberta/pytorch_model.bin
tokenizer config file saved in tokenizer-roberta/tokenizer_config.json
Special tokens file saved in tokenizer-roberta/special_tokens_map.json


('tokenizer-roberta/tokenizer_config.json',
 'tokenizer-roberta/special_tokens_map.json',
 'tokenizer-roberta/tokenizer.json')